In [163]:
import cohere

client = cohere.ClientV2("MfZwS1plvJfM7vARPs92RbCScEwRniTcCXfmfAdU", base_url="https://stg.api.cohere.ai")

PROMPT = """
Original Sentence: {raw_sentene}\n
Translated Sentence: {translated_sentene}\n

Rephrase the translated sentence to enhance its quality, ensuring it aligns closely with the original in meaning, structure, tone, and style.
Ensure that the rephrased sentence conveys the same meaning as the original sentence but avoid altering the core message or introducing new information. 
Correct any grammatical errors present in the translated sentence.
Maintain a structure similar to the original sentence. 
Match the tone and style of the original sentence. 
Preserve any stylistic elements such as enumeration, punctuation or capitalization.

The output must strictly follow this format:\n
Rephrase Translated Sentence: <sentence>"""

import json
import os
import random

In [164]:
key = "RecapCauldronLocalized_narratives_translation"

In [165]:
eng_Latn_sample = []
with open(f"/home/olivernan_cohere_com/recap_cauldron_translation_2024_10_11_raw/{key}/eng_Latn/train.jsonl", "r") as f:
    i = 0
    for line in f:
        eng_Latn_sample.append(json.loads(line))
        i += 1
        if i == 10:
            break
    # eng_Latn = [json.loads(line) for line in f]

In [166]:
samples = []
for sample in eng_Latn_sample:
    example = {"id": sample["command_id"], "User":[], "Chatbot":[]}
    for turn in sample["turns"]:
        if turn["role"] == "User":
            for content in turn["content"]:
                if 'text' in content:
                    example["User"].append({"text": content["text"], "language": "eng_Latn", "source": "raw"})
        if turn["role"] == "Chatbot":
            example["Chatbot"].append({"text": turn["content"], "language": "eng_Latn", "source": "raw-gpt_recap"})
    samples.append(example)

In [167]:
samples_dict = {sample["id"]: sample for sample in samples}

In [168]:
for lang in os.listdir(f"/home/olivernan_cohere_com/recap_cauldron_translation_2024_10_11_raw/{key}/"):
    if lang == "eng_Latn":
        continue
    with open(f"/home/olivernan_cohere_com/recap_cauldron_translation_2024_10_11_raw/{key}/{lang}/train.jsonl", "r") as f:
        for line in f:
            data = json.loads(line)
            if data['command_id'] in samples_dict:
                for turn in data["turns"]:
                    if turn["role"] == "User":
                        for content in turn["content"]:
                            if 'text' in content:
                                samples_dict[data['command_id']]['User'].append({"text": content["text"], "language": lang, "source": "raw-nllb_transl"})
                    if turn["role"] == "Chatbot":
                        samples_dict[data['command_id']]["Chatbot"].append({"text": turn["content"], "language": lang, "source": "raw-gpt_recap-nllb_transl"})

samples = list(samples_dict.values())

In [169]:
import re
dataset = []
from tqdm import tqdm
def match(output):
    return re.search(r'Rephrase Translated Sentence:\s*(.+)', output)

for sample in samples:
    for user in sample['User']:
        if user["language"] == "eng_Latn":
            eng_Latn_user = user["text"]
            break
    for chatbot in sample['Chatbot']:
        if chatbot["language"] == "eng_Latn":
            eng_Latn_chatbot = chatbot["text"]
            break
    
    
    for i, user in enumerate(tqdm(sample['User'])):
        example = {}
        example['index'] = i
        language = user["language"]
        if language != "eng_Latn":
            
            formatted_prompt = PROMPT.format(raw_sentene=eng_Latn_user, translated_sentene=user["text"])
            example['User'] = eng_Latn_user
            example['User-Translated'] = user["text"]
            
            # response = client.chat(
            #     model="command-r-plus",
            #     messages=[
            #         {
            #             "role": "user",
            #             "content": formatted_prompt
            #         }
            #     ],
            #     temperature = 0.7,
            #     p = 0.9,
            # )

            # output = match(response.message.content[0].text)
            # if output:
            #     example['User-Translated-Rephrase'] = output.group(1).strip()
            # else:
            #     example['User-Translated-Rephrase'] = response.message.content[0].text

            for bot in sample['Chatbot']:
                if bot["language"] == language:
                    formatted_prompt = PROMPT.format(raw_sentene=eng_Latn_chatbot, translated_sentene=bot["text"])
                    example['Chatbot'] = eng_Latn_chatbot
                    example['Chatbot-Translated'] = bot["text"]
                    # response = client.chat(
                    #     model="command-r-plus",
                    #     messages=[
                    #         {
                    #             "role": "user",
                    #             "content": formatted_prompt
                    #         }
                    #     ],
                    #     temperature = 0.7,
                    #     p = 0.9,
                    # )

                    # output = match(response.message.content[0].text)
                    # if output:
                    #     example['Chatbot-Translated-Rephrase'] = output.group(1).strip()
                    # else:
                    #     example['Chatbot-Translated-Rephrase'] = response.message.content[0].text
                    break
            example['language'] = user["language"]
            dataset.append(example)

100%|██████████| 24/24 [00:00<00:00, 82241.25it/s]


In [171]:
client = cohere.ClientV2("Ymj9fYrCh1uXyttUrxFppAPCzFKkB3jz6ELJcx2i")
for i, data in enumerate(tqdm(dataset)):
    eng_Latn_user = data['User']
    eng_Latn_chatbot = data['Chatbot']
    translated_user = data['User-Translated']
    translated_chatbot = data['Chatbot-Translated']

    formatted_prompt = PROMPT.format(raw_sentene=eng_Latn_user, translated_sentene=translated_user)
    response = client.chat(
        model="command-r-plus",
        messages=[
            {
                "role": "user",
                "content": formatted_prompt
            }
        ],
        temperature = 0.7,
        p = 0.9,
    )
    # print(response)
    output = match(response.message.content[0].text)
    if output:
        dataset[i]['User-Translated-Rephrase'] = output.group(1).strip()
    else:
        dataset[i]['User-Translated-Rephrase'] = response.message.content[0].text
    
    formatted_prompt = PROMPT.format(raw_sentene=eng_Latn_chatbot, translated_sentene=translated_chatbot)
    response = client.chat(
        model="command-r-plus",
        messages=[
            {
                "role": "user",
                "content": formatted_prompt
            }
        ],
        temperature = 0.7,
        p = 0.9,
    )
    # print(response)
    output = match(response.message.content[0].text)
    if output:
        dataset[i]['Chatbot-Translated-Rephrase'] = output.group(1).strip()
    else:
        dataset[i]['Chatbot-Translated-Rephrase'] = response.message.content[0].text
    # break

 14%|█▍        | 33/230 [08:49<52:43, 16.06s/it]


ReadTimeout: The read operation timed out

In [159]:
import datasets
datasets.Dataset.from_list(dataset).push_to_hub("olivernan/cauldron_nllb_translate_rephrase", key, split="train")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.52it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase/commit/903efeb095b60e4cf17ff72ffcb7f6d3fc045348', commit_message='Upload dataset', commit_description='', oid='903efeb095b60e4cf17ff72ffcb7f6d3fc045348', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase', endpoint='https://huggingface.co', repo_type='dataset', repo_id='olivernan/cauldron_nllb_translate_rephrase'), pr_revision=None, pr_num=None)

In [173]:

column_reorder = ['index', 'language', 'User', 'User-Translated', 'User-Translated-Rephrase', 'Chatbot', 'Chatbot-Translated', 'Chatbot-Translated-Rephrase']

datasets.load_dataset("olivernan/cauldron_nllb_translate_rephrase", 'RecapCauldronCocoqa_translation')['train'].select_columns(column_reorder).push_to_hub("olivernan/cauldron_nllb_translate_rephrase", 'RecapCauldronLocalized_narratives_translation', split="train")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase/commit/f22ebec64ff455cedc2317f90e0a9facd1b04069', commit_message='Upload dataset', commit_description='', oid='f22ebec64ff455cedc2317f90e0a9facd1b04069', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase', endpoint='https://huggingface.co', repo_type='dataset', repo_id='olivernan/cauldron_nllb_translate_rephrase'), pr_revision=None, pr_num=None)

In [51]:
import datasets

import cohere

# client = cohere.ClientV2("MfZwS1plvJfM7vARPs92RbCScEwRniTcCXfmfAdU", base_url="https://stg.api.cohere.ai")
client = cohere.ClientV2("Ymj9fYrCh1uXyttUrxFppAPCzFKkB3jz6ELJcx2i")

dataset = datasets.load_dataset("olivernan/cauldron_nllb_translate_rephrase", 'RecapCauldronCocoqa_translation')['train']

In [52]:
dataset

Dataset({
    features: ['index', 'language', 'User', 'User-Translated', 'User-Translated-Rephrase', 'Chatbot', 'Chatbot-Translated', 'Chatbot-Translated-Rephrase'],
    num_rows: 230
})

In [53]:
PROMPT = """Original Sentence: {raw_sentene}\n
Translated Sentence: {translated_sentene}\n
Rephrase the translation to improve its quality, ensuring it aligns closely with the original input in meaning, structure, tone, and style.
Maintain the core message without introducing new information. 
Include all sentences from the original input.
Correct any grammatical errors in the translated sentence.
Preserve the original sentence structure as much as possible. 
Match the tone and style of the original sentence. 
Retain any stylistic elements such as enumeration, punctuation, or capitalization.\n
The output must strictly follow this format:
Rephrase Translated Sentence: <sentence>"""

In [54]:
from tqdm import tqdm
import re
import time
new_dataset = []

def make_request(prompt):
    i = 0
    while i < 20:
        try:
            response = client.chat(
                model="command-r-plus-08-2024",
                messages=[
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature = 0.7,
                p = 0.9,
            )
            return response.message.content[0].text
        except Exception as e:
            i += 1
            print(e)
            time.sleep(2)
            continue

    # return response.message.content[0].text

    # output= re.search(r'Rephrase Translated Sentence:\s*(.+)', response.message.content[0].text) 
    # output_chatbot = re.search(r'Rephrase Translated Sentence:\s*(.+)', response_chatbot.message.content[0].text)
    
    #     except Exception as e:
    #         i += 1
    #         print(e)
    #         print(response)
    #         continue
    # return None
    
for data in tqdm(dataset):
    formatted_input_user = PROMPT.format(raw_sentene=data['User'], translated_sentene=data['User-Translated'])
    formatted_input_chatbot = PROMPT.format(raw_sentene=data['Chatbot'], translated_sentene=data['Chatbot-Translated'])
    i = 0
    output_user = make_request(formatted_input_user)
    output_chatbot = make_request(formatted_input_chatbot)   
    while not re.search(r'Rephrase Translated Sentence:\s*(.+)', output_user):
        output_user = make_request(formatted_input_user)
        i += 1
        print(f"User retry: {i}")
        if i == 1:
            print(output_user)
        if i == 20:
            break
    i = 0
    while not re.search(r'Rephrase Translated Sentence:\s*(.+)', output_chatbot):
        output_chatbot = make_request(formatted_input_chatbot)
        i += 1
        print(f"Chatbot retry: {i}")
        if i == 1:
            print(output_chatbot)
        if i == 20:
            break

    data['User-Translated-Rephrase-2'] = re.search(r'Rephrase Translated Sentence:\s*(.+)', output_user).group(1).strip() 
    data['Chatbot-Translated-Rephrase-2'] = re.search(r'Rephrase Translated Sentence:\s*(.+)', output_chatbot).group(1).strip()
    # print(data['User-Translated-Rephrase-2'])
    # print(data['Chatbot-Translated-Rephrase-2'])
    # break
    new_dataset.append(data)
        


 15%|█▍        | 34/230 [02:29<11:07,  3.41s/it]

The read operation timed out


 25%|██▌       | 58/230 [09:04<11:34,  4.04s/it]  

User retry: 1
Rephrase Translated Sentence: İstasyonda yolcuların binmesini bekliyor.


 45%|████▍     | 103/230 [11:41<04:58,  2.35s/it]

...

 83%|████████▎ | 191/230 [21:42<02:15,  3.46s/it]

The read operation timed out


 93%|█████████▎| 215/230 [28:26<01:13,  4.91s/it]

The read operation timed out
The read operation timed out


100%|██████████| 230/230 [39:41<00:00, 10.35s/it] 


In [55]:
new_dataset = datasets.Dataset.from_list(new_dataset)
new_dataset

Dataset({
    features: ['index', 'language', 'User', 'User-Translated', 'User-Translated-Rephrase', 'Chatbot', 'Chatbot-Translated', 'Chatbot-Translated-Rephrase', 'User-Translated-Rephrase-2', 'Chatbot-Translated-Rephrase-2'],
    num_rows: 230
})

In [57]:
column_reorder = ['index', 'language', 'User', 'User-Translated', 'User-Translated-Rephrase','User-Translated-Rephrase-2', 'Chatbot', 'Chatbot-Translated', 'Chatbot-Translated-Rephrase', 'Chatbot-Translated-Rephrase-2']
new_dataset = new_dataset.select_columns(column_reorder)

new_dataset.push_to_hub("olivernan/cauldron_nllb_translate_rephrase", 'RecapCauldronCocoqa_translation', split="train")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase/commit/e380273d83286e28bfe24415bd67ec5152a8760b', commit_message='Upload dataset', commit_description='', oid='e380273d83286e28bfe24415bd67ec5152a8760b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase', endpoint='https://huggingface.co', repo_type='dataset', repo_id='olivernan/cauldron_nllb_translate_rephrase'), pr_revision=None, pr_num=None)

In [59]:
# load dataset from parquet huggingface
datasets.load_dataset('parquet', data_files='/home/olivernan_cohere_com/RecapCauldronTextcaps_translation.parquet')['train'].push_to_hub("olivernan/cauldron_nllb_translate_rephrase", 'RecapCauldronTextcaps_translation', split="train")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.96it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase/commit/c8bf4d7d226f93981e443e97319b7667a559c30b', commit_message='Upload dataset', commit_description='', oid='c8bf4d7d226f93981e443e97319b7667a559c30b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/olivernan/cauldron_nllb_translate_rephrase', endpoint='https://huggingface.co', repo_type='dataset', repo_id='olivernan/cauldron_nllb_translate_rephrase'), pr_revision=None, pr_num=None)